# Offline Section: Quiz 4 - Problem 2

## Linear Programming Model Formulation

This problem corresponds to the type of integer programming model known as the Set Covering problem.

In a Set Covering problem, the goal is to find a subset of the columns (in this case, routes) of a given binary matrix (in this case, the presence or absence of a distribution center in a route) that covers all the rows (in this case, meeting the demands of all distribution centers) at a minimum cost (in this case, the total length of the selected routes).

In the context of this problem, each distribution center must be covered at least once by the selected routes, and the goal is to minimize the total length of the routes, which constitutes the cost. This makes it a Set Covering problem.

Firstly, let's understand the comparison you made:

The previous problem you shared about assigning landing slots to airlines is indeed a type of combinatorial optimization problem, and more specifically, a kind of Knapsack Problem, which is a common packing problem. The goal was to maximize the total bid price while ensuring each slot is assigned at most once and each airline is awarded at most one bid.

The constraints in the airline problem:

- Each slot can be assigned to at most one bid.
- Each airline can win at most one bid.

These constraints are comparable to the Ohadi Foods routing problem, which also needs to ensure:

- Each distribution center is included in at least one route.
- The total length of the routes used is minimized.

Despite the similarities, the specific requirements and constraints for each problem will influence the formulation and solution approach.

Now, let's see how we can formulate the Ohadi Foods routing problem in the same way we did for the airline problem.

The problem we are dealing with here involves a logistics company that wants to determine the optimal routes for delivering products from a central depot to various distribution centers.

There are certain constraints in this scenario:
- Each truck has a maximum carrying capacity of 30,000lbs.
- Each distribution center's demand must be satisfied by a single truck.
- The demand for each distribution center cannot be split among multiple trucks.
- The goal is to minimize the total distance traveled, since the costs are directly dependent on this.

Based on historical data, there are 43 possible truck routes that can be used to deliver the demand. Each route has its length and the distribution centers it covers.

Now, let's consider your example of the logical rule, and apply it to our problem. In our case, we have a similar situation with binary variables, but the logical constraints are slightly different. Our binary variables are whether or not a route is taken (1 if taken, 0 if not). The constraint we need to satisfy is that each distribution center's demand is met by at least one route.

***Let us denote:***

$R$ as the set of possible routes. 

$C$ as the set of distribution centers. 

$d_c$ as the demand for each center $c \in C$.

$LR_{rc}$ as an indicator variable for each route $r \in R$ and each center $c \in C$. It equals 1 if route $r$ includes center $c$, and 0 otherwise.

$l_r$ as the length of each route $r \in R$.

$x_r$ as a binary decision variable which equals 1 if route $r$ is selected, and 0 otherwise.

**Objective Function:**

We want to minimize the total length of all routes:

$$\min \sum_{r \in R} l_r \cdot x_r$$

**Constraints:**

1. `Covering Constraint:` For each distribution center, the sum of routes that include the center must be greater or equal to 1. This ensures that every center's demand is met:

$$\sum_{r \in R} LR_{rc} \cdot x_r \geq 1, \quad \forall c \in C$$

2. `Binary Constraint:` The decision variables $x_r$ are binary:

$$x_r \in \{0, 1\}, \quad \forall r \in R$$

Here we are looking for a selection of routes that covers all distribution centers with a minimum total route length.


Let us denote:

- $R$ as the set of all routes.
- $C$ as the set of all distribution centers.
- $LR_{rc}$ as the binary parameter that is 1 if route $r \in R$ includes center $c \in C$, and 0 otherwise.
- $d_c$ as the demand at each center $c \in C$.
- $l_r$ as the length of each route $r \in R$.
- $x_r$ as the binary decision variable which equals 1 if route $r$ is selected, 0 otherwise, for $r \in R$.

**Objective Function:**

We want to minimize the total length of all routes:

$$\min \sum_{r \in R} l_r \cdot x_r$$

**Constraints:**

1. `Coverage Constraint:` For each distribution center, the sum of routes that include the center must be greater or equal to 1. This ensures that every center's demand is met:

$$\sum_{r \in R} LR_{rc} \cdot x_r \geq 1, \quad \forall c \in C$$

2. `Binary Constraint:` The decision variables $x_r$ are binary:

$$x_r \in \{0, 1\}, \quad \forall r \in R$$

In this problem, we aim to minimize the total length of the routes that meet all the demand at each distribution center while respecting the constraints.


Let's denote:

- $R$ as the set of all possible routes.
- $D$ as the set of all distribution centers.
- $L_{r}$ as the length of route $r$, for $r \in R$.
- $x_{r}$ as the binary decision variable which equals 1 if route $r$ is selected, 0 otherwise, for $r \in R$.
- $I_{rd}$ equals 1 if distribution center $d$ is included in route $r$, and 0 otherwise, for $r \in R$, $d \in D$.

**Objective Function:**

Minimize the total length of the selected routes:

$$\min \sum_{r \in R} L_{r} \cdot x_{r}$$

**Constraints:**

1. `Covering Constraint:` Each distribution center must be included in at least one of the selected routes.

$$\sum_{r \in R: I_{rd} = 1} x_{r} \ge 1 \quad \forall d \in D$$

This formulation aims to minimize the total length of the routes selected to serve all the distribution centers. Each distribution center must be covered by at least one route.


## Python Implementation

In [2]:
import csv

# open and read the csv file
f = open("quiz4_routing.csv")
csvfile = csv.DictReader(f, delimiter=',')
headers = csvfile.fieldnames

table = []
for row in csvfile:
    table.append(row)
    
f.close()

# define the set of routes
Routes = headers[:]
Routes.remove('Distribution Center')
Routes.remove('Demand (lbs)')

# read the parameters
Demand = {}        # Demand[j] is the demand for distribution center j
IsInRoute = {}     # IsInRoute[i,j] = 1 if route i includes distribution center j
Length = {}        # Length[i] is the length of route i
for row in table:
    dc = row['Distribution Center']
    if dc != 'n/a':
        Demand[dc] = float(row['Demand (lbs)'])
        for r in Routes:
            if row[r] != '':
                IsInRoute[(r,dc)] = int(row[r])
    else:
        for r in Routes:
            Length[r] = float(row[r])
DCs = Demand.keys()

In [3]:
from docplex.mp.model import Model

# create a model
mdl = Model()

In [4]:
# variables
x = mdl.binary_var_dict(Routes, name='Route')

In [5]:
# objective
TotalLength = mdl.sum(Length[i]*x[i] for i in Routes)
mdl.minimize(TotalLength)

In [6]:
# constraints: covering
for j in DCs:
    mdl.add_constraint( mdl.sum(IsInRoute.get((i,j), 0)*x[i] for i in Routes) >= 1 )

In [7]:
# KPIs
mdl.add_kpi(TotalLength, 'Total length of routes')
mdl.add_kpi(mdl.sum(x[i] for i in Routes), 'Number of routes used')

DecisionKPI(name=Number of routes used,expr=Route_1+Route_2+Route_3+Route_4+Route_5+Route_6+Route_7+Route_8+..)

In [8]:
# solve
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0.094,status='integer optimal solution')

In [9]:
# print the selected routes
print("\nThe selected routes are:")
for i in Routes:
    if x[i].solution_value > 0.5:
        print(f"Route {i}")

# print total length
print("\nThe total length of the selected routes is:", mdl.objective_value)


The selected routes are:
Route 5
Route 6
Route 9
Route 17
Route 24
Route 26
Route 27

The total length of the selected routes is: 7922.0


In [10]:
# print KPIs
mdl.report()

# calculate and print the average length of the routes used
num_routes = mdl.kpi_value_by_name('Number of routes used')
if num_routes > 0:
    print("The average length of the routes used is:", mdl.objective_value / num_routes)

* model docplex_model1 solved with objective = 7922.000
*  KPI: Total length of routes = 7922.000
*  KPI: Number of routes used  = 7.000
The average length of the routes used is: 1131.7142857142858


In [11]:
from docplex.mp.model import Model

# create a model
mdl = Model('OhadiFoods')

# variables
x = mdl.binary_var_dict(Routes, name='Route')

# objective
TotalLength = mdl.sum(Length[r]*x[r] for r in Routes)
mdl.minimize(TotalLength)

# constraints: covering
for d in DCs:
    mdl.add_constraint( mdl.sum(IsInRoute.get((r,d), 0)*x[r] for r in Routes) >= 1 )

# solve
mdl.solve()

# print the selected routes
print("\nThe selected routes are:")
for r in Routes:
    if x[r].solution_value > 0.5:
        print(f"Route {r}")



The selected routes are:
Route 5
Route 6
Route 9
Route 17
Route 24
Route 26
Route 27
